## SQL Queries - Simple

### Tasks

For every customer with a 'Main Office' in Dallas show AddressLine1 of the 'Main Office' and AddressLine1 of the 'Shipping' address - if there is no shipping address leave it blank. Use one row per customer.

In [ ]:
-- sql here

For each order show the SalesOrderID and SubTotal calculated three ways:

* From the SalesOrderHeader
* Sum of OrderQty * UnitPrice
* Sum of OrderQty * ListPrice

In [ ]:
-- sql here

Show the best selling item by value.

In [ ]:
-- sql here

Show how many orders are in the following ranges (in $):

```
  RANGE      Num Orders      Total Value
    0-  99
  100- 999
 1000-9999
10000-
```

In [ ]:
-- sql here

Identify the three most important cities. Show the break down of top level product category against city.

In [ ]:
-- sql here

### Notes & Conclusion

Notes are here.